In [ ]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import imageio
import glob
import PIL
import nibabel as nib
from tkinter import Tcl
import cv2
from scipy import ndimage
import shutil

# CROPPING THE VOLUMES 
# Define your cropping function
def crop_image(data):
    # Find the min and max non-zero slices along each axis
    coords = np.array(np.where(data))
    min_coords = coords.min(axis=1)
    max_coords = coords.max(axis=1) + 1  # we want the slice after the last non-zero slice

    # Crop the image
    cropped_data = data[min_coords[0]:max_coords[0],
                        min_coords[1]:max_coords[1],
                        min_coords[2]:max_coords[2]]

    return cropped_data, min_coords, max_coords

# Path to the data
data_path = 'path to data' # Save original copy of your data since this will override the original volumes
coords_path = 'Path to save the crop coordinates' # will be used later to restore predictions to original shape

# Create the coords directory if it doesn't exist
os.makedirs(coords_path, exist_ok=True)
#!rm -r "/content/data/BraTS2021_train/.ipynb_checkpoints" #Uncomment this only when using  
# Iterate over each patient's folder
for patient_folder in os.listdir(data_path):
    patient_path = os.path.join(data_path, patient_folder)
    print(patient_path)
    # Get the list of modalities
    modalities = [f for f in os.listdir(patient_path) if f.endswith('.nii.gz')]
    print(modalities)
    # Load the first modality
    img = nib.load(os.path.join(patient_path, modalities[0]))

    # Get the data as a numpy array
    data = img.get_fdata()

    # Crop the image
    cropped_data, min_coords, max_coords = crop_image(data)

    # Save the crop coordinates
    np.save(os.path.join(coords_path, f'{patient_folder}_crop_coords.npy'), np.array([min_coords, max_coords]))

    # Iterate over each modality
    for modality in modalities:
        modality_path = os.path.join(patient_path, modality)
        print(modality_path)
        # Load the image
        img = nib.load(modality_path)

        # Get the data as a numpy array
        data = img.get_fdata()

        # Crop the image using the same coordinates
        cropped_data = data[min_coords[0]:max_coords[0],
                            min_coords[1]:max_coords[1],
                            min_coords[2]:max_coords[2]]

        # Save the cropped image
        cropped_img = nib.Nifti1Image(cropped_data, img.affine)
        nib.save(cropped_img, modality_path)

# SEPERATING THE VOLUMES INTO IMAGES
data_list = sorted(glob.glob(data_path + '/*'))            #list of paths of the inside files
results_path = 'Path to save images'                         #after the cropping of the volumes

for path in data_list:
    path_num = os.path.basename(path)
    new_result_path = os.path.join(results_path, path_num)

    new_T1_path = os.path.join(new_result_path, "images", "GIF T1", "T1_GIF")
    new_T2_path = os.path.join(new_result_path, "images", "GIF T2", "T2_GIF")
    new_Flair_path = os.path.join(new_result_path, "images", "GIF Flair", "Flair_GIF")
    
    if not os.path.exists(new_T1_path):
        os.makedirs(new_T1_path)

    if not os.path.exists(new_T2_path):
        os.makedirs(new_T2_path)

    if not os.path.exists(new_Flair_path):
        os.makedirs(new_Flair_path)

            
    modules_list = sorted(glob.glob(path + '/*'))  
    flair_vol_path = modules_list[0]                      #path of flair volume is always the first ('sorted')
    t1_vol_path = modules_list[2]                         #path of T1 volume is always the third ('sorted')
    t2_vol_path = modules_list[4]                         #path of T2 volume is always the fifth ('sorted')

    flair_nifti = nib.load(flair_vol_path).get_fdata()                    #Flair volume
    T1_nifti = nib.load(t1_vol_path).get_fdata()                          #T1 volume
    T2_nifti = nib.load(t2_vol_path).get_fdata()                          #T2 volume
    

    for i in range(T1_nifti.shape[2]):
        flair_nifti_slice = flair_nifti[:, :, i]
        t1_nifti_slice = T1_nifti[:, :, i]
        t2_nifti_slice = T2_nifti[:, :, i]

        # Normalize to [0, 1]
        flair_nifti_slice = (flair_nifti_slice - flair_nifti_slice.min()) / (flair_nifti_slice.max() - flair_nifti_slice.min())
        t1_nifti_slice = (t1_nifti_slice - t1_nifti_slice.min()) / (t1_nifti_slice.max() - t1_nifti_slice.min())
        t2_nifti_slice = (t2_nifti_slice - t2_nifti_slice.min()) / (t2_nifti_slice.max() - t2_nifti_slice.min())
    
        # Scale to [0, 255]
        flair_nifti_slice = (flair_nifti_slice * 255).astype(np.uint8)
        t1_nifti_slice = (t1_nifti_slice * 255).astype(np.uint8)
        t2_nifti_slice = (t2_nifti_slice * 255).astype(np.uint8)
        
        flair_rslt_path = f"{new_Flair_path}/{i:03d}.png"
        t1_rslt_path = f"{new_T1_path}/{i:03d}.png"
        t2_rslt_path = f"{new_T2_path}/{i:03d}.png"

        imageio.imwrite(flair_rslt_path, flair_nifti_slice)
        imageio.imwrite(t1_rslt_path, t1_nifti_slice)
        imageio.imwrite(t2_rslt_path, t2_nifti_slice)